<a href="https://colab.research.google.com/github/taniajasmin/AI-Powered-Recommendation-System/blob/main/Movie_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# dataset link = https://www.kaggle.com/datasets/carolzhangdc/imdb-5000-movie-dataset

In [ ]:
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

# Assuming the uploaded file is named 'movie_dataset.xlsx'
# If it has a different name, replace 'movie_dataset.xlsx' with the actual filename
file_name = list(uploaded.keys())[0]

Saving movie_metadata.csv to movie_metadata.csv


In [ ]:
# Loading the CSV file into a pandas DataFrame
df = pd.read_csv(file_name)

print("First 5 records:")
print(df.head())

First 5 records:
   color      director_name  num_critic_for_reviews  duration  \
0  Color      James Cameron                   723.0     178.0   
1  Color     Gore Verbinski                   302.0     169.0   
2  Color         Sam Mendes                   602.0     148.0   
3  Color  Christopher Nolan                   813.0     164.0   
4    NaN        Doug Walker                     NaN       NaN   

   director_facebook_likes  actor_3_facebook_likes      actor_2_name  \
0                      0.0                   855.0  Joel David Moore   
1                    563.0                  1000.0     Orlando Bloom   
2                      0.0                   161.0      Rory Kinnear   
3                  22000.0                 23000.0    Christian Bale   
4                    131.0                     NaN        Rob Walker   

   actor_1_facebook_likes        gross                           genres  ...  \
0                  1000.0  760505847.0  Action|Adventure|Fantasy|Sci-Fi  ...   

In [ ]:
print("Columns in the original dataset:")
print(df.columns.tolist())

Columns in the original dataset:
['color', 'director_name', 'num_critic_for_reviews', 'duration', 'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name', 'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name', 'movie_title', 'num_voted_users', 'cast_total_facebook_likes', 'actor_3_name', 'facenumber_in_poster', 'plot_keywords', 'movie_imdb_link', 'num_user_for_reviews', 'language', 'country', 'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes', 'imdb_score', 'aspect_ratio', 'movie_facebook_likes']


Preprocess the dataset

In [ ]:
# Selecting relevant columns and rename them
relevant_columns = {
    'movie_title': 'title',
    'genres': 'genres',
    'director_name': 'director',
    'language': 'language',
    'actor_1_name': 'actors',
    'imdb_score': 'rating'
}

In [ ]:
# Create a new DataFrame with only the relevant columns
processed_df = df[relevant_columns.keys()].rename(columns=relevant_columns)

In [ ]:
# # Clean the data
# processed_df = processed_df.dropna()  # Remove rows with missing values
# processed_df['genres'] = processed_df['genres'].apply(lambda x: str(x).replace('|', ' ') if pd.notna(x) else '')  # Replace '|' with space in genres
# processed_df['actors'] = processed_df['actors'].apply(lambda x: str(x).split(' ')[0] if pd.notna(x) else '')  # Keep only the first name of the actor


processed_df.loc[:, 'genres'] = processed_df['genres'].apply(lambda x: str(x).replace('|', ' ') if pd.notna(x) else '')
processed_df.loc[:, 'actors'] = processed_df['actors'].apply(lambda x: str(x).split(' ')[0] if pd.notna(x) else '')

In [ ]:
# Display the processed data
print("\nProcessed data (first 5 records):")
print(processed_df.head())


Processed data (first 5 records):
                                       title                           genres  \
0                                    Avatar   Action Adventure Fantasy Sci-Fi   
1  Pirates of the Caribbean: At World's End          Action Adventure Fantasy   
2                                   Spectre         Action Adventure Thriller   
3                     The Dark Knight Rises                   Action Thriller   
5                               John Carter           Action Adventure Sci-Fi   

            director language     actors  rating  
0      James Cameron  English        CCH     7.9  
1     Gore Verbinski  English     Johnny     7.1  
2         Sam Mendes  English  Christoph     6.8  
3  Christopher Nolan  English        Tom     8.5  
5     Andrew Stanton  English      Daryl     6.6  


In [ ]:
# doenload the new csv file
processed_df.to_csv('movies.csv', index=False)

print("\nProcessed data saved to 'movies.csv'")


Processed data saved to 'movies.csv'


# Recommendation

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
class RecommendationSystem:
    def __init__(self, data_path):
        self.data = pd.read_csv(data_path)


    def filter_by_genre(self, genre):
          return self.data[self.data['genres'].str.contains(genre, case=False, na=False)]


    def filter_by_actor(self, actor):
        return self.data[self.data['actors'].str.contains(actor, case=False, na=False)]


    def filter_by_director(self, director):
        return self.data[self.data['director'].str.contains(director, case=False, na=False)]


    def filter_by_rating(self, min_rating, max_rating):
        return self.data[(self.data['rating'] >= min_rating) & (self.data['rating'] <= max_rating)]



    def display_results(self, filtered_data):
        if filtered_data.empty:
            print("No movies found matching the criteria.")
        else:
            print(f"Found {len(filtered_data)} movies:")
            for _, row in filtered_data.iterrows():
                print(f"Title: {row['title']}")
                print(f"Genres: {row['genres']}")
                print(f"Director: {row['director']}")
                print(f"Actors: {row['actors']}")
                print(f"Rating: {row['rating']}")
                print("---")

In [ ]:
if __name__ == "__main__":
    recommender = RecommendationSystem('movies.csv')

    while True:
        print("\nFilter movies by:")
        print("1. Genre")
        print("2. Actor")
        print("3. Director")
        print("4. Rating")
        print("5. Exit")

        choice = input("Enter your choice (1-5): ")

        if choice == '1':
            genre = input("Enter a genre: ")
            results = recommender.filter_by_genre(genre)
            recommender.display_results(results)
        elif choice == '2':
            actor = input("Enter an actor's name: ")
            results = recommender.filter_by_actor(actor)
            recommender.display_results(results)
        elif choice == '3':
            director = input("Enter a director's name: ")
            results = recommender.filter_by_director(director)
            recommender.display_results(results)
        elif choice == '4':
            min_rating = float(input("Enter minimum rating: "))
            max_rating = float(input("Enter maximum rating: "))
            results = recommender.filter_by_rating(min_rating, max_rating)
            recommender.display_results(results)
        elif choice == '5':
            print("Exiting the program.")
            break
        else:
            print("Invalid choice. Please try again.")


Filter movies by:
1. Genre
2. Actor
3. Director
4. Rating
5. Exit
Enter your choice (1-5): 5
Exiting the program.


# For multiple filters

In [ ]:
class RecommendationSystem:
    def __init__(self, data_path):

        self.data = pd.read_csv(data_path)

    def filter_by_genre(self, genre):

        return self.data[self.data['genres'].str.contains(genre, case=False, na=False)]

    def filter_by_actor(self, actor):

        return self.data[self.data['actors'].str.contains(actor, case=False, na=False)]

    def filter_by_director(self, director):

        return self.data[self.data['director'].str.contains(director, case=False, na=False)]

    def filter_by_language(self, language):

        return self.data[self.data['language'].str.contains(language, case=False, na=False)]

    def filter_by_rating(self, min_rating, max_rating):

        return self.data[(self.data['rating'] >= min_rating) & (self.data['rating'] <= max_rating)]

    def apply_filters(self, filters):

        filtered_data = self.data.copy()
        for filter_type, value in filters.items():
            if filter_type == 'genre':
                filtered_data = filtered_data[filtered_data['genres'].str.contains(value, case=False, na=False)]
            elif filter_type == 'actor':
                filtered_data = filtered_data[filtered_data['actors'].str.contains(value, case=False, na=False)]
            elif filter_type == 'director':
                filtered_data = filtered_data[filtered_data['director'].str.contains(value, case=False, na=False)]
            elif filter_type == 'language':
                filtered_data = filtered_data[filtered_data['language'].str.contains(value, case=False, na=False)]
            elif filter_type == 'rating':
                min_rating, max_rating = value
                filtered_data = filtered_data[(filtered_data['rating'] >= min_rating) & (filtered_data['rating'] <= max_rating)]
        return filtered_data

    def display_results(self, filtered_data):
        if filtered_data.empty:
            print("No movies found matching the criteria.")
        else:
            print(f"Found {len(filtered_data)} movies:")
            for _, row in filtered_data.iterrows():
                print(f"Title: {row['title']}")
                print(f"Genres: {row['genres']}")
                print(f"Director: {row['director']}")
                print(f"Actors: {row['actors']}")
                print(f"Language: {row['language']}")
                print(f"Rating: {row['rating']}")
                print("---")


In [ ]:
if __name__ == "__main__":

    recommender = RecommendationSystem('movies.csv')

    while True:
        print("\nHow many filters do you want to apply? (1-5, or 0 to exit)")
        num_filters = int(input("Enter number of filters: "))

        if num_filters == 0:
            print("Exiting the program.")
            break

        if num_filters < 1 or num_filters > 5:
            print("Invalid number of filters. Please enter a number between 1 and 5.")
            continue

        filters = {}
        for i in range(num_filters):
            print(f"\nFilter {i+1}:")
            print("1. Genre")
            print("2. Actor")
            print("3. Director")
            print("4. Language")
            print("5. Rating")

            filter_choice = input("Enter your choice (1-5): ")

            if filter_choice == '1':
                genre = input("Enter a genre: ")
                filters['genre'] = genre
            elif filter_choice == '2':
                actor = input("Enter an actor's name: ")
                filters['actor'] = actor
            elif filter_choice == '3':
                director = input("Enter a director's name: ")
                filters['director'] = director
            elif filter_choice == '4':
                language = input("Enter a language: ")
                filters['language'] = language
            elif filter_choice == '5':
                min_rating = float(input("Enter minimum rating: "))
                max_rating = float(input("Enter maximum rating: "))
                filters['rating'] = (min_rating, max_rating)
            else:
                print("Invalid choice. Skipping this filter.")

        results = recommender.apply_filters(filters)
        recommender.display_results(results)


How many filters do you want to apply? (1-5, or 0 to exit)
Enter number of filters: 2

Filter 1:
1. Genre
2. Actor
3. Director
4. Language
5. Rating
Enter your choice (1-5): 4
Enter a language: french

Filter 2:
1. Genre
2. Actor
3. Director
4. Language
5. Rating
Enter your choice (1-5): 5
Enter minimum rating: 6
Enter maximum rating: 7
Found 25 movies:
Title: Evolution 
Genres: Drama Horror Mystery Sci-Fi
Director: Lucile Hadzihalilovic
Actors: Nissim
Language: French
Rating: 6.4
---
Title: A Monster in Paris 
Genres: Adventure Animation Comedy Fantasy Music Romance
Director: Bibo Bergeron
Actors: Adam
Language: French
Rating: 6.8
---
Title: A Perfect Plan 
Genres: Comedy Romance
Director: Pascal Chaumeil
Actors: Dany
Language: French
Rating: 6.3
---
Title: Coco Before Chanel 
Genres: Biography Drama
Director: Anne Fontaine
Actors: Alessandro
Language: French
Rating: 6.7
---
Title: Bon voyage 
Genres: Comedy Drama Mystery Romance Thriller War
Director: Jean-Paul Rappeneau
Actors: Isa

# Final

In [ ]:
class RecommendationSystem:
    def __init__(self, data_path):

        self.data = pd.read_csv(data_path)

    def filter_by_genre(self, genre):

        return self.data[self.data['genres'].str.contains(genre, case=False, na=False)]

    def filter_by_actor(self, actor):

        return self.data[self.data['actors'].str.contains(actor, case=False, na=False)]

    def filter_by_director(self, director):

        return self.data[self.data['director'].str.contains(director, case=False, na=False)]

    def filter_by_language(self, language):

        return self.data[self.data['language'].str.contains(language, case=False, na=False)]

    def filter_by_rating(self, min_rating, max_rating):

        return self.data[(self.data['rating'] >= min_rating) & (self.data['rating'] <= max_rating)]

    def apply_filters(self, filters):

        filtered_data = self.data.copy()
        for filter_type, value in filters.items():
            if filter_type == 'genre':
                filtered_data = filtered_data[filtered_data['genres'].str.contains(value, case=False, na=False)]
            elif filter_type == 'actor':
                filtered_data = filtered_data[filtered_data['actors'].str.contains(value, case=False, na=False)]
            elif filter_type == 'director':
                filtered_data = filtered_data[filtered_data['director'].str.contains(value, case=False, na=False)]
            elif filter_type == 'language':
                filtered_data = filtered_data[filtered_data['language'].str.contains(value, case=False, na=False)]
            elif filter_type == 'rating':
                min_rating, max_rating = value
                filtered_data = filtered_data[(filtered_data['rating'] >= min_rating) & (filtered_data['rating'] <= max_rating)]
        return filtered_data

    def display_results(self, filtered_data):

        if filtered_data.empty:
            print("No movies found matching the criteria.")
        else:
            print(f"Found {len(filtered_data)} movies:")
            for _, row in filtered_data.iterrows():
                print(f"Title: {row['title']}")
                print(f"Genres: {row['genres']}")
                print(f"Director: {row['director']}")
                print(f"Actors: {row['actors']}")
                print(f"Language: {row['language']}")
                print(f"Rating: {row['rating']}")
                print("---")

    def get_available_options(self, column_name):

        return self.data[column_name].dropna().unique()[:10].tolist()

In [ ]:
if __name__ == "__main__":

    recommender = RecommendationSystem('movies.csv')

    while True:
        print("\nAvailable filters:")
        print("1. Genre")
        print("2. Actor")
        print("3. Director")
        print("4. Language")
        print("5. Rating")

        print("\nHow many filters do you want to apply? (1-5, or 0 to exit)")
        num_filters = int(input("Enter number of filters: "))

        if num_filters == 0:
            print("Exiting the program.")
            break

        if num_filters < 1 or num_filters > 5:
            print("Invalid number of filters. Please enter a number between 1 and 5.")
            continue

        filters = {}
        for i in range(num_filters):
            print(f"\nFilter {i+1}:")
            filter_choice = input("Enter your choice (1-5): ")

            if filter_choice == '1':
                genre = input("Enter a genre: ")
                filters['genre'] = genre
            elif filter_choice == '2':
                print("Available actors (first 10):")
                for actor in recommender.get_available_options('actors'):
                    print(f"- {actor}")
                actor = input("Enter an actor's name: ")
                filters['actor'] = actor
            elif filter_choice == '3':
                director = input("Enter a director's name: ")
                filters['director'] = director
            elif filter_choice == '4':
                print("Available languages (first 10):")
                for language in recommender.get_available_options('language'):
                    print(f"- {language}")
                language = input("Enter a language: ")
                filters['language'] = language
            elif filter_choice == '5':
                min_rating = float(input("Enter minimum rating: "))
                max_rating = float(input("Enter maximum rating: "))
                filters['rating'] = (min_rating, max_rating)
            else:
                print("Invalid choice. Skipping this filter.")

        results = recommender.apply_filters(filters)
        recommender.display_results(results)


Available filters:
1. Genre
2. Actor
3. Director
4. Language
5. Rating

How many filters do you want to apply? (1-5, or 0 to exit)
Enter number of filters: 2

Filter 1:
Enter your choice (1-5): 1
Enter a genre: horror

Filter 2:
Enter your choice (1-5): 4
Available languages (first 10):
- English
- Japanese
- French
- Mandarin
- Aboriginal
- Spanish
- Filipino
- Hindi
- Russian
- Maya
Enter a language: japanese
Found 3 movies:
Title: Godzilla Resurgence 
Genres: Action Adventure Drama Horror Sci-Fi
Director: Hideaki Anno
Actors: Mark
Language: Japanese
Rating: 8.2
---
Title: Godzilla Resurgence 
Genres: Action Adventure Drama Horror Sci-Fi
Director: Hideaki Anno
Actors: Mark
Language: Japanese
Rating: 8.2
---
Title: The Cure 
Genres: Crime Horror Mystery Thriller
Director: Kiyoshi Kurosawa
Actors: Kôji
Language: Japanese
Rating: 7.4
---

Available filters:
1. Genre
2. Actor
3. Director
4. Language
5. Rating

How many filters do you want to apply? (1-5, or 0 to exit)
Enter number of fi

In [ ]:
if __name__ == "__main__":

    recommender = RecommendationSystem('movies.csv')

    while True:
        print("\nAvailable filters:")
        print("1. Genre")
        print("2. Actor")
        print("3. Director")
        print("4. Language")
        print("5. Rating")

        print("\nHow many filters do you want to apply? (1-5, or 0 to exit)")
        num_filters = int(input("Enter number of filters: "))

        if num_filters == 0:
            print("Exiting the program.")
            break

        if num_filters < 1 or num_filters > 5:
            print("Invalid number of filters. Please enter a number between 1 and 5.")
            continue

        filters = {}
        for i in range(num_filters):
            print(f"\nFilter {i+1}:")
            filter_choice = input("Enter your choice (1-5): ")

            if filter_choice == '1':
                print("Available genres (first 10):")
                for genre in recommender.get_available_options('genres'):
                    print(f"- {genre}")
                genre = input("Enter a genre: ")
                filters['genre'] = genre
            elif filter_choice == '2':
                print("Available actors (first 10):")
                for actor in recommender.get_available_options('actors'):
                    print(f"- {actor}")
                actor = input("Enter an actor's name: ")
                filters['actor'] = actor
            elif filter_choice == '3':
                print("Available directors (first 10):")
                for director in recommender.get_available_options('director'):
                    print(f"- {director}")
                director = input("Enter a director's name: ")
                filters['director'] = director
            elif filter_choice == '4':
                print("Available languages (first 10):")
                for language in recommender.get_available_options('language'):
                    print(f"- {language}")
                language = input("Enter a language: ")
                filters['language'] = language
            elif filter_choice == '5':
                print("Rating filter: Enter a range between 1 and 10.")
                min_rating = float(input("Enter minimum rating: "))
                max_rating = float(input("Enter maximum rating: "))
                filters['rating'] = (min_rating, max_rating)
            else:
                print("Invalid choice. Skipping this filter.")

        results = recommender.apply_filters(filters)
        recommender.display_results(results)

Streaming output truncated to the last 5000 lines.
Director: Peter MacDonald
Actors: Sylvester
Language: English
Rating: 5.7
---
Title: Resident Evil: Afterlife 
Genres: Action Adventure Horror Sci-Fi
Director: Paul W.S. Anderson
Actors: Milla
Language: English
Rating: 5.9
---
Title: Red Tails 
Genres: Action Adventure Drama History War
Director: Anthony Hemingway
Actors: David
Language: English
Rating: 5.9
---
Title: DragonHeart 
Genres: Action Adventure Fantasy
Director: Rob Cohen
Actors: Dennis
Language: English
Rating: 6.4
---
Title: After the Sunset 
Genres: Action Comedy Crime Drama
Director: Brett Ratner
Actors: Salma
Language: English
Rating: 6.3
---
Title: Ghost Rider: Spirit of Vengeance 
Genres: Action Fantasy Thriller
Director: Mark Neveldine
Actors: Nicolas
Language: English
Rating: 4.3
---
Title: The Pacifier 
Genres: Action Comedy Drama Family Thriller
Director: Adam Shankman
Actors: Vin
Language: English
Rating: 5.5
---
Title: Walking Tall 
Genres: Action Crime
Director

# Final final


In [ ]:
import pandas as pd

class RecommendationSystem:
    def __init__(self, data_path):

        self.data = pd.read_csv(data_path)

    def filter_by_genre(self, genre):

        return self.data[self.data['genres'].str.contains(genre, case=False, na=False)]

    def filter_by_actor(self, actor):

        return self.data[self.data['actors'].str.contains(actor, case=False, na=False)]

    def filter_by_director(self, director):

        return self.data[self.data['director'].str.contains(director, case=False, na=False)]

    def filter_by_language(self, language):

        return self.data[self.data['language'].str.contains(language, case=False, na=False)]

    def filter_by_rating(self, min_rating, max_rating):

        return self.data[(self.data['rating'] >= min_rating) & (self.data['rating'] <= max_rating)]

    def apply_filters(self, filters):

        filtered_data = self.data.copy()
        for filter_type, value in filters.items():
            if filter_type == 'genre':
                filtered_data = filtered_data[filtered_data['genres'].str.contains(value, case=False, na=False)]
            elif filter_type == 'actor':
                filtered_data = filtered_data[filtered_data['actors'].str.contains(value, case=False, na=False)]
            elif filter_type == 'director':
                filtered_data = filtered_data[filtered_data['director'].str.contains(value, case=False, na=False)]
            elif filter_type == 'language':
                filtered_data = filtered_data[filtered_data['language'].str.contains(value, case=False, na=False)]
            elif filter_type == 'rating':
                min_rating, max_rating = value
                filtered_data = filtered_data[(filtered_data['rating'] >= min_rating) & (filtered_data['rating'] <= max_rating)]
        return filtered_data

    def display_results(self, filtered_data):

        if filtered_data.empty:
            print("No movies found matching the criteria.")
        else:
            print(f"Found {len(filtered_data)} movies:")
            for _, row in filtered_data.iterrows():
                print(f"Title: {row['title']}")
                print(f"Genres: {row['genres']}")
                print(f"Director: {row['director']}")
                print(f"Actors: {row['actors']}")
                print(f"Language: {row['language']}")
                print(f"Rating: {row['rating']}")
                print("---")

    def get_unique_genres(self):

        genres = self.data['genres'].dropna().str.split(' ')
        unique_genres = set(genre for sublist in genres for genre in sublist)
        return list(unique_genres)[:10]

    def get_available_options(self, column_name):

        return self.data[column_name].dropna().unique()[:10].tolist()

if __name__ == "__main__":

    recommender = RecommendationSystem('movies.csv')

    while True:
        print("\nAvailable filters:")
        print("1. Genre")
        print("2. Actor")
        print("3. Director")
        print("4. Language")
        print("5. Rating")

        print("\nHow many filters do you want to apply? (1-5, or 0 to exit)")
        num_filters = int(input("Enter number of filters: "))

        if num_filters == 0:
            print("Exiting the program.")
            break

        if num_filters < 1 or num_filters > 5:
            print("Invalid number of filters. Please enter a number between 1 and 5.")
            continue

        filters = {}
        for i in range(num_filters):
            print(f"\nFilter {i+1}:")
            filter_choice = input("Enter your choice (1-5): ")

            if filter_choice == '1':
                print("Available genres (first 10):")
                for genre in recommender.get_unique_genres():
                    print(f"- {genre}")
                genre = input("Enter a genre: ")
                filters['genre'] = genre
            elif filter_choice == '2':
                print("Available actors (first 10):")
                for actor in recommender.get_available_options('actors'):
                    print(f"- {actor}")
                actor = input("Enter an actor's name: ")
                filters['actor'] = actor
            elif filter_choice == '3':
                print("Available directors (first 10):")
                for director in recommender.get_available_options('director'):
                    print(f"- {director}")
                director = input("Enter a director's name: ")
                filters['director'] = director
            elif filter_choice == '4':
                print("Available languages (first 10):")
                for language in recommender.get_available_options('language'):
                    print(f"- {language}")
                language = input("Enter a language: ")
                filters['language'] = language
            elif filter_choice == '5':
                print("Rating filter: Enter a range between 1 and 10.")
                min_rating = float(input("Enter minimum rating: "))
                max_rating = float(input("Enter maximum rating: "))
                filters['rating'] = (min_rating, max_rating)
            else:
                print("Invalid choice. Skipping this filter.")

        results = recommender.apply_filters(filters)
        recommender.display_results(results)


Available filters:
1. Genre
2. Actor
3. Director
4. Language
5. Rating

How many filters do you want to apply? (1-5, or 0 to exit)
Enter number of filters: 1

Filter 1:
Enter your choice (1-5): 1
Available genres (first 10):
- Thriller
- Musical
- Horror
- Biography
- Romance
- Adventure
- Documentary
- Short
- Film-Noir
- Music
Enter a genre: short
Found 5 movies:
Title: The Touch 
Genres: Romance Short
Director: Jane Clark
Actors: Necar
Language: English
Rating: 5.2
---
Title: Dolphins and Whales 3D: Tribes of the Ocean 
Genres: Adventure Documentary Short
Director: Jean-Jacques Mantello
Actors: Charlotte
Language: English
Rating: 6.5
---
Title: Marilyn Hotchkiss' Ballroom Dancing and Charm School 
Genres: Comedy Family Musical Romance Short
Director: Randall Miller
Actors: William
Language: English
Rating: 7.1
---
Title: Vessel 
Genres: Horror Sci-Fi Short Thriller
Director: Clark Baker
Actors: Taylor
Language: English
Rating: 6.2
---
Title: Jesus People 
Genres: Comedy Short
Direc